Load dataset

In [ ]:
# %%capture
# !pip install ultralytics
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="Ql1dOvGZp8vqTAnCTntx")
# project = rf.workspace("kiyong-kang-qhti3").project("plants-2l8n0")
# dataset = project.version(8).download("yolov8")

Import libraries

In [15]:
import os
import shutil
import copy
import glob
import random

Create tip burn directory

In [2]:
if not os.path.exists('./Plants-tipburn/train/images'):
    os.makedirs('./Plants-tipburn/train/images')

if not os.path.exists('./Plants-tipburn/train/labels'):
    os.makedirs('./Plants-tipburn/train/labels')

if not os.path.exists('./Plants-tipburn/valid/images'):
    os.makedirs('./Plants-tipburn/valid/images')

if not os.path.exists('./Plants-tipburn/valid/labels'):
    os.makedirs('./Plants-tipburn/valid/labels')

In [3]:
t_labels = os.listdir('./Plants-8/train/labels')
v_labels = os.listdir('./Plants-8/valid/labels')

train_src = './Plants-8/train'
val_src = './Plants-8/valid'

train_dir = './Plants-tipburn/train'
val_dir = './Plants-tipburn/valid'

Filter tipburn files for object detection

In [8]:
def filter_tipburn_det(src, labels):
    det_files = []

    for label_name in labels:
        filename = label_name[:-4]
        label_dir = src + '/labels/' + filename + '.txt'
        with open(label_dir, 'r') as f:
            objs = f.readlines()

        n_obj_tipburn = 0
        objs_tipburn = []
        objs_others = []

        for obj in objs:
            if obj[0] == '0':
                n_obj_tipburn += 1
                objs_tipburn.append(obj)
            else:
                objs_others.append(obj)

        if n_obj_tipburn == len(objs):
            filename_tipburn = filename + '.tipburn'
            label_dir_tipburn = src + '/labels/' + filename_tipburn + '.txt'
            image_dir = src + '/images/' + filename + '.jpg'
            image_dir_tipburn = src + '/images/' + filename_tipburn + '.jpg'
            det_files.append(filename_tipburn)
            os.rename(label_dir, label_dir_tipburn)
            os.rename(image_dir, image_dir_tipburn)
        elif n_obj_tipburn < len(objs) and n_obj_tipburn > 0:
            filename_tipburn = filename + '.tipburn'
            label_dir_tipburn = src + '/labels/' + filename_tipburn + '.txt'
            image_dir = src + '/images/' + filename + '.jpg'
            image_dir_tipburn = src + '/images/' + filename_tipburn + '.jpg'
            det_files.append(filename_tipburn)
            shutil.copy(image_dir, image_dir_tipburn)
            with open(label_dir_tipburn, 'w') as f2:
                f2.writelines(objs_tipburn)
            with open(label_dir, 'w') as f3:
                f3.writelines(objs_others)
        elif n_obj_tipburn == 0:
            continue

    return det_files

In [9]:
det_t_files = filter_tipburn_det(train_src, t_labels)
det_v_files = filter_tipburn_det(val_src, v_labels)

In [10]:
print('Number of train data: ', len(t_labels))
print('Number of train tip burn data: ', len(det_t_files))
print('Number of valid data: ', len(v_labels))
print('Number of valid valid data: ', len(det_v_files))

Number of train data:  1458
Number of train tip burn data:  384
Number of valid data:  133
Number of valid valid data:  4


Move tip burn images/labels for object detection

In [11]:
def move_files(src, dir, files):
    for filename in files:
        shutil.move(src + '/images/' + filename + '.jpg', dir + '/images/' + filename + '.jpg')
        shutil.move(src + '/labels/' + filename + '.txt', dir + '/labels/' + filename + '.txt')

In [12]:
move_files(train_src, train_dir, det_t_files)
move_files(val_src, val_dir, det_v_files)

In [13]:
print('Number of train data: ', len(os.listdir(train_src + '/images')), '/', len(os.listdir(train_src + '/labels')))
print('Number of valid data: ', len(os.listdir(val_src + '/images')), '/', len(os.listdir(val_src + '/labels')))
print('Number of tipburn train images/labels: ', len(os.listdir(train_dir + '/images')), '/', len(os.listdir(train_dir + '/labels')))
print('Number of tipburn valid images/labels: ', len(os.listdir(val_dir + '/images')), '/', len(os.listdir(val_dir + '/labels')))

Number of train data:  1443 / 1443
Number of valid data:  133 / 133
Number of tipburn train images/labels:  384 / 384
Number of tipburn valid images/labels:  4 / 4


Increase validation dataset

In [18]:
x = os.listdir(train_dir + '/images')
x_val = random.sample(x, 6)
x_val = [i[:-4] for i in x_val]

move_files(train_dir, val_dir, x_val)

In [19]:
print('Number of tipburn train images/labels: ', len(os.listdir(train_dir + '/images')), '/', len(os.listdir(train_dir + '/labels')))
print('Number of tipburn valid images/labels: ', len(os.listdir(val_dir + '/images')), '/', len(os.listdir(val_dir + '/labels')))

Number of tipburn train images/labels:  378 / 378
Number of tipburn valid images/labels:  10 / 10


Modify labels in Plants-8
- butterhead lettuce: 1 -> 0
- romaine lettuce: 2 -> 1

In [16]:
def modify_labels(dir):
    files = glob.glob(dir)
    for file in files:
        with open(file, 'r') as f:
            labels = f.readlines()
        for i, label in enumerate(labels):
            if label[0] == '1':
                labels[i] = '0' + label[1:]
            elif label[0] == '2':
                labels[i] = '1' + label[1:]

        with open(file, 'w') as f:
            f.writelines(labels)

In [17]:
modify_labels('./Plants-8/train/labels/*.txt')
modify_labels('./Plants-8/valid/labels/*.txt')